In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [4]:
import transformers
print(transformers.__version__)

4.45.2


In [5]:
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig

from helpers import *

### Load base model and test on questions

In [6]:
USE_GPU = False

questions = [
    "Give me an 1-sentence introduction of LLM.",
    "Calculate 1+1-1",
    "What's the difference between thread and process?"
]

In [7]:
model_name = "../models/HuggingFaceTB/SmolLM"
model, tokenizer = load_model_and_tokenizer(model_name, USE_GPU)

In [8]:
train_dataset = load_dataset("banghua/DL-SFT-Dataset")["train"]
if not USE_GPU:
    train_dataset=train_dataset.select(range(100))

display_dataset(train_dataset)

,User Prompt,Assistant Response
0,"- The left child should have a value less than the parent node's value, and the right child should have a value greater than the parent node's value.","This statement is correct. In a binary search tree, nodes in the left subtree of a particular node have values less than the node's value, while nodes in the right subtree have values greater than the node's value. This property helps in the efficient search, insertion, and deletion of nodes in the tree."
1,"To pass three levels must be the plan.\nThen tackle Two, when that is done.\nOf 100 that start, at the end will be 20.\nFinQuiz is a website that helps you prepare.\nUse it to be stress-free, and not lose your hair.\nThen, take the exam with a smile on your face.\nBe confident that you will gain your place.\nSo make this the goal to which you aspire. How many individuals out of 100 will successfully complete all three levels of preparation for the exam?","Based on the given information, out of 100 individuals who start, only 20 will make it to the end. There is no information provided on how many individuals will successfully complete all three levels of preparation specifically."
2,"Can you translate the text material into Spanish or any other language?: He really is, you know.\nThings a hero should show.\nHe loves me more than a zillion things.\nHe loves me when I sing my jolly folktale rhymes.\nHe's good, not just good, in fact he's great!\nBut because he's my best mate!\nWOW !!! I love it!!!!","¿Puede traducir el texto a español o a cualquier otro idioma?: \nRealmente lo es, ya sabes.\nCosas que un héroe debería demostrar.\nMe quiere más que un millón de cosas.\nMe quiere cuando canto mis alegres rimas de cuentos populares.\nEs bueno, no solo bueno, ¡de hecho es genial!\n¡Pero porque es mi mejor amigo!\n¡WOW! ¡Me encanta!"


In [14]:
# SFTTrainer config 
sft_config = SFTConfig(
    output_dir="./sft_model",      # where to save checkpoints
    learning_rate=8e-5, # Learning rate for training. 
    num_train_epochs=1, #  Set the number of epochs to train the model.
    per_device_train_batch_size=1, # Batch size for each device (e.g., GPU) during training. 
    gradient_accumulation_steps=8, # Number of steps before performing a backward/update pass to accumulate gradients.
    gradient_checkpointing=False, # Enable gradient checkpointing to reduce memory usage during training at the cost of slower training speed.
    logging_steps=2,  # Frequency of logging training progress (log every 2 steps).

)

In [17]:
def formatting_func(batch):
    """
    Converts a batch of message dictionaries into tokenized input_ids
    for SFTTrainer. Each message should have 'role' and 'content'.
    """
    input_ids = []

    # Iterate through examples in the batch
    for messages in batch["messages"]:
        text = ""
        for message in messages:
            role = message.get("role", "")
            content = message.get("content", "")
            
            if role == "user":
                text += f"### Human:\n{content}\n"
            elif role == "assistant":
                text += f"### Assistant:\n{content}\n"
            else:
                # For any other roles, just include the content
                text += f"{content}\n"

        # Tokenize the combined text for this example
        tokenized = tokenizer(text, truncation=True, padding=False)
        input_ids.append(tokenized["input_ids"])

    return {"input_ids": input_ids}


sft_trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    formatting_func=formatting_func
)
sft_trainer.train()


Map:   0%|          | 0/100 [00:00<?, ? examples/s]


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).